In [1]:
#dependencies
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# store filepath in a variable
pev_path = "./pev_sales.xlsx"
pev_df = pd.io.excel.read_excel(pev_path,header =2, usecols = 'B:J', skipfooter=11)

In [3]:
# Reformatting the df using .melt 
pdf = pev_df.melt(id_vars=['Vehicle', 'Type'], var_name = 'Year', value_name = 'Sales')

In [4]:
# store filepath in a variable
hev_path = "./hev_sales.xlsx"
hev_df = pd.io.excel.read_excel(hev_path, header=2, usecols = 'B:BP', skipfooter=19) 

In [5]:
#Reformat df by using .melt function
hdf = hev_df.melt(id_vars=['Year'], var_name = 'Vehicle', value_name = 'Sales')
# type_hdf = hdf['Type']='HEV'
# type_hdf.head()
hdf.head()

,Year,Vehicle,Sales
0,1999,Volkswagen Touareg Hybrid,NaN
1,2000,Volkswagen Touareg Hybrid,NaN
2,2001,Volkswagen Touareg Hybrid,NaN
3,2002,Volkswagen Touareg Hybrid,NaN
4,2003,Volkswagen Touareg Hybrid,NaN


In [6]:
# Split the vehicle into make and model
new_pdf=pdf[['Make', 'Model']]=pdf.Vehicle.str.split(' ',1,expand=True)
pdf['Make'] = new_pdf[0]
pdf['Model'] = new_pdf[1]
pdf.head()

,Vehicle,Type,Year,Sales,Make,Model
0,Audi A3 Plug In,PHEV,2011,NaN,Audi,A3 Plug In
1,BMW Active E,EV,2011,NaN,BMW,Active E
2,BMW i3,EV,2011,NaN,BMW,i3
3,BMW i8,PHEV,2011,NaN,BMW,i8
4,BMW X5,PHEV,2011,NaN,BMW,X5


In [7]:
# remove the vehicle column 
del pdf['Vehicle']

In [8]:
# reordering hdf
reorg_pdf=pdf[["Make", "Model", "Type", "Year", "Sales"]]
new_PEV_df = reorg_pdf.dropna().head()

In [9]:
# Split the vehicle into make and model
new_hdf=hdf[['Make', 'Model']]=hdf.Vehicle.str.split(' ',1,expand=True)
new_hdf.head()
hdf['Make'] = new_hdf[0]
hdf['Model'] = new_hdf[1]
hdf.head()

,Year,Vehicle,Sales,Make,Model
0,1999,Volkswagen Touareg Hybrid,NaN,Volkswagen,Touareg Hybrid
1,2000,Volkswagen Touareg Hybrid,NaN,Volkswagen,Touareg Hybrid
2,2001,Volkswagen Touareg Hybrid,NaN,Volkswagen,Touareg Hybrid
3,2002,Volkswagen Touareg Hybrid,NaN,Volkswagen,Touareg Hybrid
4,2003,Volkswagen Touareg Hybrid,NaN,Volkswagen,Touareg Hybrid


In [10]:
#add the Type column in df with value HEV
hdf['Type']='HEV'

In [11]:
# remove the vehicle column 
del hdf['Vehicle']

In [12]:
#reorganize the data and drop the not a number values from the the sales column
reorg_hdf=hdf[['Make','Model', 'Type', 'Year', 'Sales']]
cleaned_hdf=reorg_hdf.dropna().head()

In [13]:
# find unique values, make label corrections with .replace
# reorg_hdf['Make'].unique()
# reorg_pdf['Make'].unique()
new_HEV_df = cleaned_hdf.replace('Acrua','Acura').replace('Volkswagen','VW')

In [19]:
#start connection into proseql
rds_connection_string = "postgres:OliverRocks33@!@localhost:5432/EV_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [20]:
#load the csv converted to Dataframe into sql db
new_HEV_df.to_sql(name='new_HEV_df', con=engine, if_exists='append', index=False)
new_PEV_df.to_sql(name='new_PEV_df', con=engine, if_exists='append', index=False)